<a href="https://colab.research.google.com/github/dgo2dance/qwen-tune/blob/main/LLaMA-Factory-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check GPU configuration
!nvidia-smi

Wed Jan 24 05:31:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#STEP 0 確認有使用GPU
##記事本必須在GPU模式下運行，可以執行上方的nvidia-smi查看GPU是否有GPU。若没有，點擊左上角的 編輯>筆記本設定，把硬體加速器改成GPU。
##如記事本有出問題可在以下儲存庫回報
> https://github.com/ADT109119/LLaMA-Factory-Colab

In [2]:
#@title STEP 1 複製儲存庫並安裝必要的函式庫
#@markdown #STEP 1
#@markdown ##複製儲存庫並安裝必要的函式庫
#@markdown ##Clone repository & Install requirements lib

!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd ./LLaMA-Factory
!pip install kaleido
!pip install -r requirements.txt
!pip install bitsandbytes>=0.39.0

#用比較殘暴的方式開啟 Gradio 的分享連結
train_web_py_file_path = "./src/train_web.py"
try:
  with open(train_web_py_file_path, 'r') as file:
    file_content = file.read()
  modified_content = file_content.replace("share=False", "share=True")
  with open(train_web_py_file_path, 'w') as file:
    file.write(modified_content)
except Exception as e:
    print(f'ERROR: {str(e)}')

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 6726, done.
remote: Counting objects: 100% (745/745), done.
remote: Compressing objects: 100% (300/300), done.
remote: Total 6726 (delta 485), reused 653 (delta 443), pack-reused 5981
Receiving objects: 100% (6726/6726), 204.47 MiB | 17.21 MiB/s, done.
Resolving deltas: 100% (4836/4836), done.
Updating files: 100% (139/139), done.
/content/LLaMA-Factory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 50.2 MB/s eta 0:0

In [3]:
! pip install cohere  openai tiktoken
! pip install transformers_stream_generator
! pip install transformers==4.36.2
! git lfs install
# !git clone https://huggingface.co/Qwen/Qwen-1_8B
!git clone https://huggingface.co/Qwen/Qwen-1_8B-Chat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.8 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.1
    Uninstalling importlib-metadata-7.0.1:
      Successfully uninstalled importlib-metadata-7.0.1
  Preparing metadata (setup.py) ... done
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.4-py3-none-any.whl size=12316 sha256=f8c7028de7b6a86fe55f63a1610f513a2e5d6bf499d79c9a0c5c41f50c390b8a
  Stored in directory: /root/.cache/pip/wheels/47/1d/3c/92d88493ed40c0d9be60a391eb76c9a56e9f9b7542cb789401
Successfully built transformers_stream_generator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 30.2 MB/s eta 0:00:00
  Attemp

In [ ]:
#@title STEP 2 開啟WebUI
#@markdown #STEP 2
#@markdown ##開啟WebUI
#@markdown ##Run the WebUI
%cd /content/LLaMA-Factory

!CUDA_VISIBLE_DEVICES=0 python src/train_web.py

/content/LLaMA-Factory
2024-01-24 05:36:01.494019: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 05:36:01.494072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 05:36:01.495416: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 05:36:02.751093: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Running on local URL:  http://0.0.0.0:7860
Running on public URL: https://5c716d1f6706ab3eec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Termi

In [ ]:
%cd /content/Qwen-1_8B-Chat
! ls

/content/Qwen-1_8B-Chat
assets				   generation_config.json	     qwen_generation_utils.py
cache_autogptq_cuda_256.cpp	   LICENSE			     qwen.tiktoken
cache_autogptq_cuda_kernel_256.cu  model-00001-of-00002.safetensors  README.md
config.json			   model-00002-of-00002.safetensors  tokenization_qwen.py
configuration_qwen.py		   modeling_qwen.py		     tokenizer_config.json
cpp_kernels.py			   model.safetensors.index.json
examples			   NOTICE


In [ ]:
! pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.26.1
aiofiles                         23.2.1
aiohttp                          3.9.1
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
b

#STEP 3 上傳輸出的檔案到 HuggingFace


In [ ]:
#@title STEP 3.1 安裝HuggingFace API
#@markdown #STEP 3.1
#@markdown ##為了上傳模型至HuggingFace，需要先安裝HF的函式庫
#@markdown ##In order to upload the model to HuggingFace, we have to install the HF library first.
!pip install huggingface_hub

In [ ]:
#@title STEP 3.2 使用API Token登入
#@markdown #STEP 3.2
#@markdown ##執行本儲存格後填入 API Token(需要有 Write 權限)，然後按下登入
#@markdown ##After runing this cell, fill in the API Token (Write permission is required), and then click Login

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#@title STEP 3.3 上傳模型
#@markdown #STEP 3.3
#@markdown ##執行本儲存格後填入 API Token(需要有 Write 權限)，然後按下登入
#@markdown ##After runing this cell, fill in the API Token (Write permission is required), and then click Login
from huggingface_hub import HfApi
api = HfApi()

model_id = "username/fine-tune-model" #@param {type:"string"}
export_folder_path = "/content/LLaMA-Factory/export" #@param {type:"string"}
api.create_repo(model_id, private=True, exist_ok=True, repo_type="model")
api.upload_folder(
    folder_path=export_folder_path,
    repo_id=model_id,
)

#STEP 4 保存到Google Drive(自選)

In [ ]:
#@title STEP 4.1 連接Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title STEP 4.2 複製檔案到Google Drive
Google_Drive_Folder = "/content/drive/MyDrive/LLaMA-Factory/" #@param {type:"string"}

!cp -rf /content/LLaMA-Factory/saves {Google_Drive_Folder}